The **Sequential model** makes the assumption that the network has exactly one input and exactly one output, and that it consists of a linear stack of layers.  

But this set of assumptions is too inflexible in a number of cases. Some networks require **several independent inputs** (**Multi-input Models**), others require **multiple outputs**, and some networks have internal branching between layers that makes them look like **graphs of layers** rather than linear stacks of layers.

## Introduction to the functional API

In the functional API, you directly manipulate tensors, and you use layers as functions that take tensors and return tensors (hence, the name functional API):

In [1]:
from keras import Input, layers

input_tensor = Input(shape=(32, ))              # a tensor

dense = layers.Dense(32, activation='relu')     # a layer is a function

output_tensor = dense(input_tensor)             # a layer may be called on a tensor, and it returns a tensor

Using TensorFlow backend.


In [2]:
# Sequential vs Functional API
from keras.models import Sequential, Model
from keras import layers
from keras import Input

# Sequential
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64, )))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# Functional equivalent
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)          # the Model class turns an input tensor and output tensor into a model

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [3]:
seq_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


The only part that may seem a bit magical at this point is instantiating a Model object using only an input tensor and an output tensor. Behind the scenes, Keras retrieves every layer involved in going from input_tensor to output_tensor, bringing them together into a graph-like data structure—a Model. Of course, the reason it works is that output_tensor was obtained by repeatedly transforming input_tensor. If you tried to build a model from inputs and outputs that weren’t related, you’d get a RuntimeError:

In [4]:
unrelated_input = Input(shape=(32,))
bad_model = model = Model(unrelated_input, output_tensor)

RuntimeError: Graph disconnected: cannot obtain value for tensor Tensor("input_2:0", shape=(?, 64), dtype=float32) at layer "input_2". The following previous layers were accessed without issue: []

In [5]:
# the compiling, evaluating such an instance of Model is the same

# compile
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy')

In [6]:
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

# train
model.fit(x_train, y_train, epochs=10, batch_size=128)

# evaluate
score = model.evaluate(x_train, y_train)

Epoch 1/10
1000/1000 [==============================] - 1s 973us/step - loss: 11.6558
Epoch 2/10
1000/1000 [==============================] - 0s 43us/step - loss: 11.5403
Epoch 3/10
1000/1000 [==============================] - 0s 40us/step - loss: 11.5204
Epoch 4/10
1000/1000 [==============================] - 0s 38us/step - loss: 11.5101
Epoch 5/10
1000/1000 [==============================] - 0s 34us/step - loss: 11.5031
Epoch 6/10
1000/1000 [==============================] - 0s 43us/step - loss: 11.4976
Epoch 7/10
1000/1000 [==============================] - 0s 36us/step - loss: 11.4924
Epoch 8/10
1000/1000 [==============================] - 0s 35us/step - loss: 11.4892
Epoch 9/10
1000/1000 [==============================] - 0s 32us/step - loss: 11.4870
Epoch 10/10
1000/1000 [==============================] - 0s 64us/step


## Multi-input Models

A question-answering model. Such models usually have two inputs: a natural-language question and a test snippet (such as a news article) providing information to be used for answerining the question.  

The model must then produce an answer: in the simplest possible setup, this is a one-word answer obtained via a softmax over some predefined vocabulary.

Following is an example of how you can build such a model with the functional API. You set up two independent branches, encoding the text input and the question input as representation vectors; then, concatenate these vectors; and finally, add a softmax classifier on top of the concatenated representations.

In [14]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500


text_input = Input(shape=(None,),                                            
                   dtype='int32', 
                   name='text')                                        # the text input is a variable-length sequence of integers. Note that you can optionally name the inputs.
embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input) # embeds the inputs into a sequence of vectors of size 64
encoded_text = layers.LSTM(32)(embedded_text)                          # encodes the vectors in a single vector via an LSTM

question_input = Input(shape=(None,), 
                       dtype='int32',
                      name='question')                                 # Same process (with different layer instance) for the question
embedded_question = layers.Embedding(32, question_vocabulary_size)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], 
                                 axis=-1)                              # concatenates the encoded question and encoded text
answer = layers.Dense(answer_vocabulary_size,
                     activation='softmax')(concatenated)                # Adds a softmax classifier on top

model = Model([text_input, question_input], answer)       # At model instantiation, you specify the two inputs and the output

# compile the model
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['acc'])

There are two possible APIs: feed the model a list of Numpy arrays as inputs, or feed it a dictionary that maps input names to Numpy arrays (if you give names to your inputs).

In [15]:
# Feeding data to a multi-input model
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size,
                        size=(num_samples, max_length))                   # Generates dummy Numpy data
question = np.random.randint(1, question_vocabulary_size,
                            size=(num_samples, max_length))
answers = np.random.randint(0, 1, 
                           size=(num_samples, answer_vocabulary_size))    # answers are one-hot encoded, not integers

## Note: Fix the bellow cells later with the help of Keras Documentaion

https://keras.io/getting-started/functional-api-guide/

In [16]:
# fitting using a list of inputs
model.fit([text, question], answers, epochs=10, batch_size=128)

Epoch 1/10


InvalidArgumentError: indices[0,0] = 4234 is not in [0, 64)
	 [[Node: embedding_6/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_6/embeddings/read, _arg_question_2_0_2)]]

Caused by op 'embedding_6/Gather', defined at:
  File "/Users/araks/anaconda/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/araks/anaconda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/araks/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/araks/anaconda/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/araks/anaconda/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/araks/anaconda/lib/python3.5/site-packages/z

In [11]:
# fitting using a dictionary of inputs (only if inputs are named)
model.fit({'text': text, 'question': question}, answers,
         epochs=10, batch_size=128)

Epoch 1/10


InvalidArgumentError: indices[0,0] = 4740 is not in [0, 32)
	 [[Node: embedding_2/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_2/embeddings/read, _arg_question_0_2)]]

Caused by op 'embedding_2/Gather', defined at:
  File "/Users/araks/anaconda/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/araks/anaconda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/araks/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/araks/anaconda/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/araks/anaconda/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/araks/anaconda/lib/python3.5/site-packages/zmq